In [1]:
# %% 
import numpy as np
import pandas as pd
import time
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
train_labeled = pd.read_csv('labled_train_data.csv', index_col=0, sep='\t', comment='#')
comments_labeled = pd.read_csv('labled_train_comments.csv', index_col=0, sep='\t', comment='#')
tracks_labeled = pd.read_csv('labled_train_tracks2.csv')

train_unlabeled = pd.read_csv('unlabled_train_data.csv', index_col=0, sep='\t', comment='#')
comments_unlabeled = pd.read_csv('unlabled_train_comments.csv', index_col=0, sep='\t', comment='#')
tracks_unlabeled = pd.read_csv('unlabled_train_tracks.csv', index_col=0, sep='\t', comment='#')

In [17]:
tracks_labeled = tracks_labeled.sort_values(['order_id', 'dt']) # sort df by time
tracks_labeled.index = range(len(tracks_labeled)) 

# Some functions for finding speed 

In [18]:
import datetime 
from math import cos, asin, sqrt, pi

# Haversine formula for calculating distances between two points
def get_distance(lat1, lon1, lat2, lon2):
    r = 6371
    phi1 = np.radians(lat1)
    phi2 = np.radians(lat2)
    delta_phi = np.radians(lat2 - lat1)
    delta_lambda = np.radians(lon2 - lon1)
    a = np.sin(delta_phi / 2)**2 + np.cos(phi1) * np.cos(phi2) *   np.sin(delta_lambda / 2)**2
    res = r * (2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a)))
    return np.round(res, 4)
    

def get_speed(lat1, lon1, lat2, lon2, dt1: str, dt2: str) -> float:
    distance = get_distance(lat1, lon1, lat2, lon2).tolist()
    format = "%Y-%m-%d %H:%M:%S"
    dt1=datetime.datetime.strptime(dt1, format)
    dt2=datetime.datetime.strptime(dt2, format)
    time = (dt2-dt1).total_seconds()/3600 # convert timedelta into hours
    if time==0:
        return 0
    return distance/time

In [19]:
import requests
import json

def get_route(order: str, df_original: pd.DataFrame) -> dict:
    route = []
    df = df_original
    df = df[df['order_id']==order]
    for _, el in df.iterrows():
        point_info = [el['lon_'], el['lat_'], el['dt']]
        route.append(point_info)
    return {order: route}

api_key = '5b3ce3597851110001cf62485e43a8ce4c9147be87344cce215d4774'
def get_gpx_from_route(route:list, api_key: str):
    route = [x[:-1] for x in route] # remove time element
    url = 'https://api.openrouteservice.org/v2/directions/driving-car/gpx'
    gpx = requests.post(url, data=json.dumps({"coordinates":route}), headers = {'Authorization': api_key, 'content-type': 'application/json'})
    return gpx

order = '001662da857b5a39bb402aacf3145f86' # just an example
route = get_route(order, tracks_labeled)
gpx = get_gpx_from_route(route[order], api_key)
with open('./route.gpx', 'w') as f:
    f.write(str(gpx.content))

In [54]:
def speed_to_df(df):
    for i in range(1, len(df)):
        df.loc[i, 'speed'] = get_speed(df.loc[i-1, 'lat_'], df.loc[i-1, 'lon_'],df.loc[i, 'lat_'], df.loc[i, 'lon_'],df.loc[i-1, 'dt'], df.loc[i, 'dt'])
    return df

speed_to_df(tracks_unlabeled).to_csv('./data/unlabled_train_tracks_speed')

In [24]:
for i,order in enumerate(train_labeled['order_id']):
    tracks_labeled.loc[tracks_labeled['order_id']==order, 'is_aggressive'] = train_labeled.loc[i, 'is_aggressive']

KeyboardInterrupt: 

,Unnamed: 0,driver_id,dt,lat_,lon_,order_id,speed
order_id,,,,,,,
001662da857b5a39bb402aacf3145f86,NaN,NaN,NaN,NaN,NaN,NaN,NaN
001662da857b5a39bb402aacf3145f86,NaN,NaN,NaN,NaN,NaN,NaN,NaN
001662da857b5a39bb402aacf3145f86,NaN,NaN,NaN,NaN,NaN,NaN,NaN
001662da857b5a39bb402aacf3145f86,NaN,NaN,NaN,NaN,NaN,NaN,NaN
001662da857b5a39bb402aacf3145f86,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
fffdfda358f35cf2d7f9d87d205655a7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fffdfda358f35cf2d7f9d87d205655a7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
fffdfda358f35cf2d7f9d87d205655a7,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
test = tracks_labeled[tracks_labeled['order_id']=='fffdfda358f35cf2d7f9d87d205655a7']

sns.lineplot(x='dt', y = 'speed', hue='order_id', data = test)

<AxesSubplot:xlabel='dt', ylabel='speed'>

1. All speed observation in one training batch.
2. Every row is different training batch.

In [8]:
# 1.
import catboost
from catboost import CatBoostClassifier
def pen(x):
    if arr[0]:
        return False
train_data =  np.delete(tracks_labeled.values, np.s_[0], 1)
train_data

AttributeError: 'numpy.ndarray' object has no attribute 'filter'